In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('/Users/scarere/Documents/UofT/OneDrive-UofT/mbp1413_final_project/code')
from utils.segment_metrics import IOU_eval
import torch
from utils.helper_functions import load_data
import numpy as np
import torch
from utils.segment_metrics import IOU_eval
import tqdm.notebook as t

In [2]:
model_path = '../trained_models/baselines/unet_basic/unet_basic_test14_3'
model_name = 'ckpt_epoch_35.pt'
data_dir = '../data'

In [3]:
model = torch.load(os.path.join(model_path, model_name), map_location=torch.device('cpu'))
model = model.eval()

In [4]:
#x_train, y_train = load_data(os.path.join(data_dir, 'data_sayan/undistorted', 'train.pt'), switch_channel_dim=True, thresh=train_args['train_mask_threshold'])
x_val, y_val = load_data(os.path.join(data_dir, 'downsized_cropped', 'val.pt'), switch_channel_dim=True, thresh=0.5)
x_test, y_test = load_data(os.path.join(data_dir, 'downsized_cropped', 'test.pt'), switch_channel_dim=True, thresh=0.5)

In [5]:
pred_val = model(x_val)

In [6]:
iou_val = IOU_eval().iou_evaluate_better(y_val.int(), pred_val)
print(float(iou_val))

0.7570582032203674


In [7]:
pred_test = model(x_test)

In [8]:
iou_test = IOU_eval().iou_evaluate_better(y_test.int(), pred_test)
print(float(iou_test))

0.7299535274505615


## Unet basic baseline averages

In [9]:
val = [0.736004, 0.776850, 0.757058]
test = [0.697885, 0.757641, 0.729954]